# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

## section 1: derivatives

In [2]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [3]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  da = -3*a**2 - 0.5*a**-0.5
  db = 3*cos(3*b) + 2.5*b**1.5
  dc = c**-2
  return [da, db, dc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [34]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO
# Grad is size of increase when size is limit to 0
h = 0.0000000001
a = 2
b = 3
c = 4

current_val = f(a,b,c)
## Grad with respect to a
new_a = a + h
nudge_a_val = f(new_a,b,c)
grad_a = (nudge_a_val-current_val)/h

## Grad with respect to b
new_b = b + h
nudge_b_val = f(a,new_b,c)
grad_b = (nudge_b_val-current_val)/h
## Grad with respect to c
new_c = c + h
nudge_c_val = f(a,b,new_c)
grad_c = (nudge_c_val-current_val)/h

numerical_grad = [grad_a, grad_b, grad_c] #
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.35355817641448
OK for dim 1: expected 10.25699027111255, yours returns 10.256986371359744
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


In [36]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# Symetric_derivative: lim h=>0 for (f(x+h) - f(x-h))/2h

# -----------
#initial data
h = 0.0000000001
a = 2
b = 3
c = 4

## Grad with respect to a
a2 = a + h
a1 = a - h
nudge_a1_val = f(a1,b,c)
nudge_a2_val = f(a2,b,c)
grad2_a = (nudge_a2_val-nudge_a1_val)/(2 * h)

## Grad with respect to b
b2 = b + h
b1 = b - h
nudge_b1_val = f(a,b1,c)
nudge_b2_val = f(a,b2,c)
grad2_b = (nudge_b2_val-nudge_b1_val)/(2 * h)

## Grad with respect to c
c2 = c + h
c1 = c - h
nudge_c1_val = f(a,b,c1)
nudge_c2_val = f(a,b,c2)
grad2_c = (nudge_c2_val-nudge_c1_val)/(2 * h)

numerical_grad2 = [grad2_a, grad2_b, grad2_c] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553735522382
OK for dim 1: expected 10.25699027111255, yours returns 10.256990812251843
OK for dim 2: expected 0.0625, yours returns 0.06250111539429781


## section 2: support for softmax

In [119]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  def __radd__(self, other):
    return self + other

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
    return out
  def __neg__(self):
    return self * -1

  def exp(self):
    x = self.data
    out = Value(exp(x),(self,), 'exp')
    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward
    return out

  def __pow__(self, x):
    assert(isinstance(x, (float, int)), 'only support float or int')
    out = Value(self.data ** x, (self,),'pow')
    def _backward():
      self.grad += (x * self.data ** (x - 1)) * out.grad
    out._backward = _backward
    return out;

  def log(self):
    out = Value(log(self.data), (self,), 'log')
    def _backward():
      self.grad += (self.data ** -1) * out.grad
    out._backward = _backward
    return out

  def __truediv__(self, other):
    return self * (other ** -1)

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

<>:58: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:58: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-119-e03d96a105ad>:58: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(isinstance(x, (float, int)), 'only support float or int')


In [151]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
print(probs)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=0.04177257051535045), Value(data=0.839024507462532), Value(data=0.00565330266221633), Value(data=0.11354961935990122)]
2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [152]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
torch_logits = torch.Tensor([0,3,-2,1]).double(); torch_logits.requires_grad = True
torch_probs = torch.softmax(torch_logits, 0)
torch_loss = -1 * torch.log(torch_probs[3])
torch_loss.backward()

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
print(torch_logits.grad)
for dim in range(4):
  ok = 'OK' if abs(torch_logits.grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {torch_logits.grad[dim]}")


tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535046
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625321
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400987
